In [46]:
import pandas as pd

# 文件路径
path_r = 'HSI_5分钟线.xlsx'
df = pd.read_excel(path_r)

# 读取数据
date = df['日期'].tolist()  # 日期
time = df['时间'].tolist()  # 时间
open_price = df['开盘'].tolist()  # 开盘价
high = df['最高'].tolist()  # 最高价
low = df['最低'].tolist()  # 最低价
close = df['收盘'].tolist()  # 收盘价
vol = df['成交量'].tolist()  # 成交量
amount = df['成交额'].tolist()  # 成交额


以下是一些建议的短期预测因子，你可以考虑将它们添加到你的模型中：

相对强弱指标（RSI）：

可以根据分钟级别的数据重新调整参数，比如使用较短的时间窗口。
指数平滑移动平均线（EMA）：

与日线相似，但可以考虑更短的时间窗口，例如5、10、15分钟的EMA。
滞后差分（Lag Difference）：

当前时刻的收盘价与前一个时刻的收盘价之间的差异，有助于捕捉短期内的趋势。
波动率指标（ATR）：

平均真实范围，用于度量价格波动性。
MACD（Moving Average Convergence Divergence）：

可以使用较短的时间窗口和较短的信号线。
威廉指标（Williams %R）：

通过测量最高价和最低价之间的相对位置来衡量市场超买或超卖状态。
价格动量指标（PPO）：

百分比价格振荡器，衡量价格相对于移动平均线的变化。
CCI（顺势指标）：

商品通道指数，用于度量价格与其统计平均数之间的偏离程度。
平均真实范围（Average True Range，ATR）：

衡量价格波动性，适用于短期预测。
DMI（动向指标）：

可以帮助确认趋势的方向和强度。
交易量加权移动平均线（VWAP）：

考虑成交量加权的价格平均值，可用于判断市场趋势。

In [47]:
# 计算每5分钟的涨跌
def calculate_5min_changes(close_prices):
    changes_5min = [0]  # 第一个时间点没有涨跌，所以设为0
    for i in range(1, len(close_prices)):
        change_5min = float(close_prices[i]) - float(close_prices[i - 1])
        changes_5min.append(change_5min)
    return changes_5min

# 计算每5分钟的涨跌幅
def calculate_5min_returns(close_prices):
    returns_5min = [0]  # 第一个时间点没有涨跌幅，所以设为0
    for i in range(1, len(close_prices)):
        prev_close_5min = float(close_prices[i - 1])
        current_close_5min = float(close_prices[i])
        return_5min = (current_close_5min - prev_close_5min) / prev_close_5min if prev_close_5min != 0 else 0
        returns_5min.append(return_5min)
    return returns_5min

# 股票每5分钟的50分钟涨幅
def Chg_ten_5min(close):
    chg_ten_5min = []
    for i in range(10, len(close)):
        c_5min = (float(close[i]) - float(close[i - 10])) / float(close[i - 10])
        chg_ten_5min.append(c_5min)
    return chg_ten_5min


# ADR是根据市场上涨和下跌的天数计算的比率
def ADR_5min(changes_5min):
    adr_5min = []
    rise = 0
    fall = 0
    for i in range(len(changes_5min)):
        if i < 10:
            if changes_5min[i] > 0:
                rise += 1
            elif changes_5min[i] < 0:
                fall += 1
            if i == 9:
                adr_ratio = round(float(rise) / fall, 4) if fall != 0 else 0
                adr_5min.append(adr_ratio)
        else:
            if changes_5min[i - 10] > 0 and changes_5min[i] < 0:
                rise -= 1
                fall += 1
            elif changes_5min[i - 10] < 0 and changes_5min[i] > 0:
                rise += 1
                fall -= 1
            adr_ratio = round(float(rise) / fall, 4) if fall != 0 else 0
            adr_5min.append(adr_ratio)
    return adr_5min


# 10日相对强弱指标(10日内涨的天数/10)
def RSI10_5min(changes_5min):
    rsi_10_5min = []
    rise = 0
    for i in range(len(changes_5min)):
        if i < 10:
            if changes_5min[i] > 0:
                rise += 1
            if i == 9:
                rsi_10_5min.append(rise / 10)
        else:
            if changes_5min[i - 10] > 0:
                rise -= 1
            if changes_5min[i] > 0:
                rise += 1
            rsi_10_5min.append(rise / 10)
    return rsi_10_5min


# 该时刻k线值=(PRICEclose−PRICEopen)/(PRICEhigh−PRICElow)
def k_value(high, low, open_price, close):
    k = []
    for i in range(len(open_price)):
        # 避免除以零
        if float(high[i]) != float(low[i]):
            kk = (float(close[i]) - float(open_price[i])) / (float(high[i]) - float(low[i]))
        else:
            # 如果最高价和最低价相同，决定如何处理
            # 这里我们可以将k值设为0，或者根据您的需求进行其他处理
            kk = 0
        k.append(kk)
    return k



# n日乖离率BIAS=（当日收盘价-N日内移动平均收盘价）/N日内移动平均收盘价
def BIAS(n, close):
    bias = []
    for i in range(n,len(close)):
        b = (close[i] - sum(close[i-n:i]) / n) / (sum(close[i-n:i]) / n)
        bias.append(b)
    return bias


# n日RSV-未成熟随机值:RSV=100x(当日收盘价–9日内最低收盘价)/(9日内最高收盘价–9日内最低收盘价)
def RSV(n, close):
    rsv = []
    for i in range(n, len(close)):
        min_price = min(close[i-n:i])
        max_price = max(close[i-n:i])

        # 避免除以零
        if min_price != max_price:
            r = (close[i] - min_price) / (max_price - min_price)
        else:
            r = 0

        rsv.append(10 * r)

    return rsv



# OBV=sign(当日股票收盘价−前日股票收盘价)×当日成交量V
def OBV(close, vol):
    obv = []
    for i in range(1, len(close)):
        x = close[i] - close[i - 1]
        if x == 0:
            sign = 0
        elif x > 0:
            sign = 1
        elif x < 0:
            sign = -1
        o = sign * vol[i]
        obv.append(o)
    return obv

def calculate_ema(data, window):
    """
    Calculate Exponential Moving Average (EMA) for a given window size.

    Parameters:
    - data: List of numerical data (e.g., close prices).
    - window: Integer, the size of the window for EMA calculation.

    Returns:
    - ema: List of EMA values.
    """
    ema = []
    multiplier = 2 / (window + 1)

    # Calculate initial EMA as the simple moving average for the first 'window' data points
    sma = sum(data[:window]) / window
    ema.append(sma)

    # Calculate EMA for the remaining data points
    for i in range(window, len(data)):
        ema_value = (data[i] - ema[-1]) * multiplier + ema[-1]
        ema.append(ema_value)

    return ema


In [48]:
#计算各个因子
changes_5min = calculate_5min_changes(close)
returns_5min = calculate_5min_returns(close)
k=k_value(high, low, open_price, close)
chg_ten_5min=Chg_ten_5min(close)
obv= OBV(close, vol)
adr_5min=ADR_5min(changes_5min)
RSI_10_5min=RSI10_5min(changes_5min)
bias=BIAS(12,close)
rsv=RSV(9,close)
ema=calculate_ema(close, 20)

l=len(k)


In [49]:

# 找到最长的因子长度
lengths = [len(changes_5min), len(returns_5min), len(k), len(chg_ten_5min), len(obv), len(adr_5min), len(RSI_10_5min), len(bias), len(rsv), len(ema)]
max_length = max(lengths)

# 将所有因子扩展到最长的长度，并用NaN填充缺失的数据
changes_5min += [float('nan')] * (max_length - len(changes_5min))
returns_5min += [float('nan')] * (max_length - len(returns_5min))
k += [float('nan')] * (max_length - len(k))
chg_ten_5min = [float('nan')] * (max_length - len(chg_ten_5min)) + chg_ten_5min
obv += [float('nan')] * (max_length - len(obv))
adr_5min = [float('nan')] * (max_length - len(adr_5min)) + adr_5min
RSI_10_5min = [float('nan')] * (max_length - len(RSI_10_5min)) + RSI_10_5min
bias = [float('nan')] * (max_length - len(bias)) + bias
rsv = [float('nan')] * (max_length - len(rsv)) + rsv
ema += [float('nan')] * (max_length - len(ema))

# 创建 DataFrame
data = {
    '日期': date[:max_length],
    '时间': time[:max_length],
    '开盘': open_price[:max_length],
    '最高': high[:max_length],
    '最低': low[:max_length],
    '收盘': close[:max_length],
    '成交量': vol[:max_length],
    '成交额': amount[:max_length],
    '5分钟涨跌': changes_5min,
    '5分钟涨跌幅': returns_5min,
    'k值': k,
    '50分钟涨幅_5分钟': chg_ten_5min,
    'OBV': obv,
    'ADR_5分钟': adr_5min,
    'RSI10_5分钟': RSI_10_5min,
    '60分钟乖离值': bias,
    '45分钟RSV': rsv,
    'EMA_20': ema
}

df = pd.DataFrame(data)
# 保存 DataFrame 到 Excel 文件
df.to_excel('HSI_5分钟线+factor.xlsx', index=False)
